In [8]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from basicfunc import *
from localsimulator import *

# 检查一个完整的模拟流程

## 1. 准备数据

In [9]:
start_date = '20220110'
end_date = '20221231'
actdays = get_datelist('20220110', '20221231', 1, -1)
iszt = loadcache(actdays[0], end_date, "ISZT1")
date_list = get_datelist('20220104', end_date, 0, 0)
isdt = loadcache(actdays[0], end_date, "ISZT")
vwap_ret = loadcache(actdays[0], end_date, "VWAPRET")
# index_ret = loadcache(actdays[0], end_date, "I500RET")
startdi = date_list.tolist().index(actdays[1])
enddi = startdi + len(actdays) - 1

In [5]:
from basefactor import fetcher
import pandas as pd
import numpy as np

vwap = fetcher.get_tradedate_data('liuhz.adjprice', '2022-01-01', '2022-12-30', 'ACTWAP')
alpha = fetcher.get_tradedate_data('liuhz.vol', '2022-01-01', '2022-12-30', 'uid')
vwap = pd.pivot_table(vwap, index='tradedate', columns='wind_code', values='ACTWAP')
alpha = pd.pivot_table(alpha, index='tradedate', columns='wind_code', values='uid')

# 清洗数据

In [28]:
vwap_1 = fetcher.get_market_data('jydb.dstock', '2022-01-01', '2022-12-30', 'VWAP')
vwap_1 = pd.pivot_table(vwap_1, index='tradedate', columns='wind_code', values='VWAP')
vwap_2 = fetcher.get_tradedate_data('jydb.dailydata', '2022-01-01', '2022-12-30', 'adjclose')
vwap_2 = pd.pivot_table(vwap_2, index='tradedate', columns='wind_code', values='adjclose')

In [37]:
np.setdiff1d(vwap_2.columns, vwap.columns)

array(['000780.SZ', '301105.SZ', '301297.SZ', '688525.SH'], dtype=object)

In [51]:
codes_map = fetcher.get_codes_map('2022-01-01', '2022-12-30')
jydb = fetcher.JY_DataFetcher(codes_map=codes_map)
vwap_4 = jydb.get_performance_data('2022-01-01', '2022-12-30', 'AvgPrice')

In [55]:
vwap_4 = pd.pivot_table(vwap_4, index='tradedate', columns='wind_code', values='avgprice')

In [56]:
vwap_4

wind_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,688787.SH,688788.SH,688789.SH,688793.SH,688798.SH,688799.SH,688800.SH,688819.SH,688981.SH,689009.SH
tradedate,,,,,,,,,,,,,,,,,,,,,
2022-01-04,16.4111,20.3188,19.9199,2.3212,4.4958,6.1980,2.9421,14.0174,3.8070,11.7319,...,99.1997,137.5728,256.0318,102.6100,209.2552,42.8440,130.8698,42.5569,52.9833,69.4800
2022-01-05,17.0445,21.1092,21.4742,2.3277,4.5208,6.2814,2.8732,13.8114,3.8364,11.7061,...,98.8928,135.7515,249.7431,100.4007,201.8428,40.2797,125.3883,41.9218,52.8865,67.8456
2022-01-06,17.1185,21.1807,21.8157,2.3187,4.5385,6.7182,2.9240,14.0252,3.8710,11.8326,...,93.6228,134.3296,248.7133,98.2094,193.2887,41.8602,127.8063,41.8030,52.8644,67.1925
2022-01-07,17.1992,21.9633,21.7066,2.3158,4.5825,6.7539,2.8942,14.0287,3.8646,11.9400,...,90.4391,131.9514,248.9021,95.4772,192.1423,42.5105,126.4427,41.8606,52.8917,65.6625
2022-01-10,17.1846,22.1010,21.4503,2.3152,4.5542,6.4695,2.8484,13.7456,3.8331,11.8747,...,85.6596,130.4207,238.7422,95.0850,190.5053,42.0269,120.3759,42.0431,52.8249,63.1690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,12.8200,18.3701,9.4061,1.8451,5.9515,7.8426,2.3092,12.6038,3.8037,11.6131,...,59.5956,36.1594,165.3674,51.7889,97.1692,33.0106,107.3735,35.8912,41.3140,32.3930
2022-12-27,13.0935,18.6199,9.5822,1.8873,6.5316,7.7341,2.3335,12.2768,3.8267,11.9875,...,59.7038,36.5773,165.5697,52.2457,95.9026,31.7236,109.5722,36.5666,41.3983,32.6165
2022-12-28,13.1751,18.5261,9.5892,1.8645,6.7613,7.8336,2.3161,12.0760,3.7735,12.0597,...,58.9020,35.8833,163.9551,51.7421,96.0313,31.7547,107.7466,36.5993,41.0783,32.2793


In [69]:
np.save('VWAP.npy', vwap_4.values())

wind_code,000001.SZ,000002.SZ,000004.SZ,000005.SZ,000006.SZ,000007.SZ,000008.SZ,000009.SZ,000010.SZ,000011.SZ,...,872392.BJ,872808.BJ,872925.BJ,873001.BJ,873122.BJ,873169.BJ,873223.BJ,873305.BJ,873339.BJ,873527.BJ
tradedate,,,,,,,,,,,,,,,,,,,,,
2022-01-04,16.411133,20.318773,19.919869,2.321243,4.495776,6.198031,2.942149,14.017441,3.806961,11.731858,...,NaN,NaN,24.140697,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-05,17.044484,21.109229,21.474204,2.327732,4.520812,6.281429,2.873240,13.811410,3.836410,11.706095,...,NaN,NaN,23.593028,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,17.118523,21.180658,21.815730,2.318656,4.538472,6.718214,2.923973,14.025174,3.870959,11.832591,...,NaN,NaN,23.695254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-07,17.199167,21.963277,21.706566,2.315847,4.582521,6.753911,2.894209,14.028653,3.864590,11.940050,...,NaN,NaN,23.543135,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-10,17.184648,22.101014,21.450335,2.315231,4.554175,6.469500,2.848358,13.745627,3.833081,11.874731,...,NaN,NaN,23.753480,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,12.819953,18.370090,9.406081,1.845075,5.951490,7.842611,2.309154,12.603786,3.803709,11.613102,...,NaN,26.955342,14.319285,NaN,11.699857,5.921648,3.326207,10.012735,5.042060,8.055051
2022-12-27,13.093507,18.619905,9.582220,1.887266,6.531615,7.734092,2.333517,12.276806,3.826665,11.987481,...,NaN,26.924363,14.337619,7.631699,11.705507,5.919597,3.330347,10.261539,5.038988,8.033539
2022-12-28,13.175084,18.526121,9.589201,1.864527,6.761272,7.833580,2.316057,12.076042,3.773530,12.059667,...,NaN,26.905018,14.210117,7.323886,11.613379,5.906843,3.297855,10.136892,4.999360,8.057285
